In [ ]:
"""
This notebook provides tools for extracting, cleaning amd geocoding
data that is publicly available on the Atlanta building efficiency website at
https://atlantabuildingefficiency.com/benchmarking-data/

The output is a CSV file wih latitude and longitudes for each building
in the database shown on that site

Note that the geocoding process can take several minutes and throw
several exceptions, but it will complete correctly as long as you have 
a proper API key for the geocoding service you are using

I welcome all help and suggestions for improvements.
Please send any to david@buildpayer.com

"""

In [4]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import os

In [2]:
import geopandas as geo
import fiona; fiona.supported_drivers
import geopy
import rtree

PROJ: proj_create_from_database: Cannot find proj.db


In [ ]:
#Enter the full location of the file in your local computer
os_path=r'C:\your_directory'

#Location of the CSV file downloaded from the Atlanta Building Efficiency Website
#https://atlantabuildingefficiency.com/benchmarking-data/
building_info_path='building_information.csv'

#Enter your Google API key for using their geocoding service
#https://developers.google.com/maps/documentation/geocoding/get-api-key
api_key='insert your key here'


In [45]:
"""
Note: Not all buildings that have efficiency data 
have an Atlanta Building ID ("ABID")
Atlanta building data available from 
https://public.tableau.com/profile/office.of.resilience#!/vizhome/shared/42RQM3P4X
This script requires download of the CSV from the site above
and does not automatically retrieve data from URL above
"""
os.chdir(os_path)


building_info = pd.read_csv(
    building_info_path,
    header=0,
    dtype={'Year Built':'int'},
    parse_dates=['Year Ending','Year'],
    infer_datetime_format=True,
    thousands=',' #removes thousands separator from bldg SF
)

In [46]:
#Find buildings where there is no Atlanta building ID 
building_info[building_info['Atlanta Building ID'].str.contains('Not Available')]
print("Number of buildings with data is "+str(len(building_info)))

#SHorten name of building square footage column for easier coding
building_info.rename(
    columns={'Property GFA - Calculated (Buildings) (ft²)':'bldgSF'},
    inplace=True
)

Number of buildings with data is 1529


In [99]:
#concatenate address city, postal code into a single item
building_info=building_info.assign(address_long=building_info['Address']+' '+building_info['City']+', GA ')

#Limit the data to 2018 metrics since the CSV has all years
building_info_2018=building_info[building_info['Year']=='2018']

#Make buildings with no Energy Star Scores zeros for that metric
building_info_2018.fillna(
    value={'ENERGY STAR Score': 0},
    inplace=True,
    downcast=True
)

C:\Users\david_000\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [29]:
import geopandas as geo
import fiona; fiona.supported_drivers
import geopy
import rtree

In [98]:
"""
This section let you use Geopandas Geolocation
tools with a Google API key
It pulls with a timeout using Google geocode API key
#source: https://geopy.readthedocs.io/en/latest/#usage-with-pandas
"""
df=building_info_2018
api_key_google=api_key

from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key=api_key_google)

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['address_long'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#Create columns for latitude and longitude
df['latitude'] = df['point'].apply(lambda x: x[0])
df['longitude'] = df['point'].apply(lambda x: x[1])

#Export geocoded addressess to a CSV file for use in mapping applications
df.to_csv(path_or_buf='./2018_geocode.csv')

RateLimiter caught an error, retrying (0/2 tries). Called with (*('120 Claire Drive Atlanta, GA ',), **{}).
Traceback (most recent call last):
  File "C:\Users\david_000\AppData\Roaming\Python\Python37\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Users\david_000\Anaconda3\lib\urllib\request.py", line 525, in open
    response = self._open(req, data)
  File "C:\Users\david_000\Anaconda3\lib\urllib\request.py", line 543, in _open
    '_open', req)
  File "C:\Users\david_000\Anaconda3\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Users\david_000\Anaconda3\lib\urllib\request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\david_000\Anaconda3\lib\urllib\request.py", line 1320, in do_open
    r = h.getresponse()
  File "C:\Users\david_000\Anaconda3\lib\http\client.py", line 1344, in getresponse
    respons

,Address,Atlanta Building ID,Blank,Views by Parameter,City,ENERGY STAR Score,Number of Buildings,Number of Records,Postal Code,Grouped Property Types,...,Total GHG Emissions (Metric Tons CO2e) (group),Total GHG Emissions (Metric Tons CO2e),Water Use (kGal),Water Use Intensity (gal/ft²),Year Built,Year Ending,Year,address_long,location,point
26,2244 Metropolitan Parkway,537284,,NaN,Atlanta,0.0,3.0,1,30315.0,"Other, Mixed Use",...,No Values Given,NaN,NaN,NaN,1975,2017-12-31,2018-01-01,"2244 Metropolitan Parkway Atlanta, GA","(2244 Metropolitan Pkwy SW, Atlanta, GA 30315,...","(33.6928905, -84.4066306, 0.0)"
27,1328 Peachtree Street NE,422029,,NaN,Atlanta,0.0,1.0,1,30309.0,"Other, Mixed Use",...,No Values Given,NaN,NaN,NaN,1965,2017-12-31,2018-01-01,"1328 Peachtree Street NE Atlanta, GA","(1328 Peachtree St NE, Atlanta, GA 30309, USA,...","(33.7906847, -84.3856753, 0.0)"
28,1500 Northside Dr. NW,558239,,NaN,Atlanta,0.0,2.0,1,30318.0,Retail/ Entertainment,...,No Values Given,NaN,NaN,NaN,1950,2017-12-31,2018-01-01,"1500 Northside Dr. NW Atlanta, GA","(1500 Northside Dr NW, Atlanta, GA 30318, USA,...","(33.7958762, -84.408813, 0.0)"
29,267 Marietta Street,421528,,NaN,Atlanta,0.0,1.0,1,30313.0,Lodging,...,No Values Given,NaN,NaN,NaN,1999,2017-12-31,2018-01-01,"267 Marietta Street Atlanta, GA","(267 Marietta St NW, Atlanta, GA 30313, USA, (...","(33.7607765, -84.39452949999999, 0.0)"
30,3030 Peachtree Rd,6354628,,NaN,Atlanta,0.0,1.0,1,30305.0,Retail/ Entertainment,...,No Values Given,NaN,NaN,NaN,2014,2017-12-31,2018-01-01,"3030 Peachtree Rd Atlanta, GA","(3030 Peachtree Rd, Atlanta, GA 30305, USA, (3...","(33.8380739, -84.3812698, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509,225 Baker Street,731349,,NaN,Atlanta,0.0,1.0,1,30313.0,"Museums, Performing Arts and Zoos",...,29259.400000000001,29259.4,52120.1,86.29,2005,2017-12-31,2018-01-01,"225 Baker Street Atlanta, GA","(225 Baker St NW, Atlanta, GA 30313, USA, (33....","(33.7638778, -84.3957609, 0.0)"
1512,1050 Techwood Drive NW,512401,,NaN,Atlanta,0.0,15.0,1,30318.0,"Other, Mixed Use",...,31034.299999999999,31034.3,45606.7,34.86,2001,2017-12-31,2018-01-01,"1050 Techwood Drive NW Atlanta, GA","(1050 Techwood Dr NW, Atlanta, GA 30318, USA, ...","(33.7829628, -84.3930896, 0.0)"
1514,56 Marietta St,375575,,1.0,Atlanta,1.0,1.0,1,30303.0,Office,...,31748.299999999999,31748.3,12678.2,77.69,1917,2017-12-31,2018-01-01,"56 Marietta St Atlanta, GA","(56 Marietta St NW, Atlanta, GA 30303, USA, (3...","(33.7554591, -84.3914507, 0.0)"
1521,1 CNN Center Plz NW,442848,,NaN,Atlanta,0.0,1.0,1,30303.0,"Other, Mixed Use",...,42261.0,42261.0,69747.1,39.08,1976,2017-12-31,2018-01-01,"1 CNN Center Plz NW Atlanta, GA","(190 Marietta St NW, Atlanta, GA 30303, USA, (...","(33.7582642, -84.39500960000001, 0.0)"


In [ ]:
"""
A list of buildings subject to the ordinance is available as an excel file at the following link:
https://web.atlantaga.gov/coabuildings/#/

This file needs cleaning up as many numeric values are coded as a string
If that is the case, use the code snippets below to clean up those values

#remove linebreaks from SF columns
building_info.replace('\n',' ', regex=True, inplace=True)

#remove whitespace around strings in SF column

building_info.replace('\n',' ', regex=True, inplace=True)
#building_info['bldgSF']=building_info['bldgSF'].str.strip()

#find non-numeric string values in SF
non_numeric=building_info['bldgSF'].str.isnumeric()

#replace non-numeric sq. ft values with NaN values
#this makes it easier to remove them
building_info['bldgSF'].where(non_numeric,inplace=True)

#convert sq ft. from strings to floating values
building_info['bldgSF']=building_info['bldgSF'].astype(float)
building_info['bldgSF'].dtype
"""